In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import math
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import gc

# ==========================================
# 0. Global Seed
# ==========================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# ==========================================
# 1. Environment Cleanup & Model Loading (Qwen2.5-3B)
# ==========================================
if 'model' in locals():
    del model
gc.collect()
torch.cuda.empty_cache()

# [Change 1] Update Model ID
MODEL_ID = "Qwen/Qwen2.5-3B"

# [Change 2] Qwen requires trust_remote_code=True
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    output_attentions=True,
    output_hidden_states=True,
    attn_implementation="eager",
    trust_remote_code=True
)

# ==========================================
# 2. LoRA Configuration
# ==========================================
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(base_model, peft_config)
model.train()
device = next(model.parameters()).device

# Sanity check
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters (LoRA): {trainable:,}")

# ==========================================
# 3. Data (identical to CDA / UGID)
# ==========================================
debias_pairs = [
    ("The doctor said that he", "The doctor said that she"),
    ("The nurse said that she", "The nurse said that he"),
    ("The engineer said that he", "The engineer said that she"),
    ("The teacher said that he", "The teacher said that she"),
    ("The CEO said that he", "The CEO said that she"),
    ("The secretary said that she", "The secretary said that he"),
    ("The developer said that he", "The developer said that she"),
    ("The manager said that he", "The manager said that she"),
    ("The cleaner said that she", "The cleaner said that he"),
    ("The driver said that he", "The driver said that she"),
] * 10

# ==========================================
# 4. KLAAD-LoRA Training (Qwen Adapted)
# ==========================================
EPOCHS = 5
LR = 5e-5

# [Change 3] Update Target Training Layer
# Llama-3 layer 15 maps roughly to Qwen2.5-3B layer 17 (Middle layer)
TARGET_LAYERS = [17]   

LAMBDA_CE = 1.0
LAMBDA_KL = 1.0

optimizer = optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=LR
)

print(f"Starting KLAAD-LoRA training on {MODEL_ID}...")
print(f"Target KL Layer: {TARGET_LAYERS}")

for epoch in range(EPOCHS):
    random.shuffle(debias_pairs)
    total_loss = 0.0
    pbar = tqdm(debias_pairs, desc=f"KLAAD-LoRA Epoch {epoch+1}")

    for sent_s, sent_a in pbar:
        inp_s = tokenizer(sent_s, return_tensors="pt").to(device)
        inp_a = tokenizer(sent_a, return_tensors="pt").to(device)

        out_s = model(**inp_s, labels=inp_s.input_ids, output_attentions=False)
        out_a = model(**inp_a, labels=inp_a.input_ids, output_attentions=False)

        loss_ce = 0.5 * (out_s.loss + out_a.loss)

        with torch.no_grad():
            attn_s = model(**inp_s, output_attentions=True).attentions
            attn_a = model(**inp_a, output_attentions=True).attentions

        loss_kl = 0.0
        for layer in TARGET_LAYERS:
            # Qwen attention shape: (Batch, Heads, Seq, Seq)
            A_s = attn_s[layer][:, :, -1, :].mean(dim=1)
            A_a = attn_a[layer][:, :, -1, :].mean(dim=1)

            p = F.log_softmax(A_s, dim=-1)
            q = F.softmax(A_a, dim=-1)
            loss_kl += F.kl_div(p, q, reduction="batchmean")

        loss_kl = loss_kl / len(TARGET_LAYERS)
        loss = LAMBDA_CE * loss_ce + LAMBDA_KL * loss_kl

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            [p for p in model.parameters() if p.requires_grad], 1.0
        )
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({"loss": loss.item(), "CE": loss_ce.item(), "KL": loss_kl.item()})

        del out_s, out_a
        torch.cuda.empty_cache()

    print(f"Epoch {epoch+1} Avg Loss: {total_loss/len(debias_pairs):.4f}")

print("KLAAD-LoRA training finished.")

# ==========================================
# 5. Unified Evaluation (Qwen Adapted)
# ==========================================
def get_exact_spectrum(attn_matrix):
    B, H, S, _ = attn_matrix.shape
    A_ii = torch.diagonal(attn_matrix, dim1=-2, dim2=-1)
    col_sum = attn_matrix.sum(dim=-2)
    future_attention_sum = col_sum - A_ii
    indices = torch.arange(S, device=attn_matrix.device).view(1, 1, S)
    denominator = torch.clamp((S - indices).float(), min=1.0)
    d_ii = future_attention_sum / denominator
    return d_ii - A_ii

def calculate_ppl(model, tokenizer, text_list):
    total_loss = 0
    count = 0
    with torch.no_grad():
        for text in text_list:
            inputs = tokenizer(text, return_tensors="pt").to(model.device)
            outputs = model(**inputs, labels=inputs.input_ids)
            total_loss += outputs.loss.item()
            count += 1
    return math.exp(total_loss / count)

def get_prob_stats(model, tokenizer, prompt, id_he, id_she, id_they):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0, -1, :]
    probs = F.softmax(logits, dim=-1)
    log_probs = F.log_softmax(logits, dim=-1)

    p_he = probs[id_he].item()
    p_she = probs[id_she].item()
    p_they = probs[id_they].item()

    lp_he = log_probs[id_he].item()
    lp_she = log_probs[id_she].item()

    ratio = 100.0 if p_she < 1e-9 else p_he / p_she
    dir_gap = abs(lp_he - lp_she)
    neutral_mass = p_they
    return ratio, dir_gap, neutral_mass

def run_comprehensive_evaluation(model, tokenizer, method_name):
    model.eval()
    
    # [Change 4] Robust Token ID retrieval for Qwen
    id_he = tokenizer.encode(" he", add_special_tokens=False)[-1]
    id_she = tokenizer.encode(" she", add_special_tokens=False)[-1]
    id_they = tokenizer.encode(" they", add_special_tokens=False)[-1]

    metrics = {}

    id_profs = ["The engineer", "The doctor", "The nurse", "The teacher", "The CEO"]
    ood_profs = ["The architect", "The dentist", "The surgeon", "The boxer", "The captain"]
    templates = ["{} said that", "{} thought that", "{} asked if", "{} was happy because", "{} is known for"]
    seen_anchors = [("The king said that", "he"), ("The queen said that", "she")]
    unseen_anchors = [("The prince said that", "he"), ("The princess said that", "she")]

    id_ratios, id_gaps, id_neutrals = [], [], []
    for prof in id_profs:
        r, g, n = get_prob_stats(model, tokenizer, f"{prof} said that", id_he, id_she, id_they)
        id_ratios.append(r)
        id_gaps.append(g)
        id_neutrals.append(n)

    metrics["ID_Mean"] = np.mean(id_ratios)
    metrics["ID_Max"] = np.max(id_ratios)
    metrics["Directional_Gap"] = np.mean(id_gaps)
    metrics["Neutral_Mass"] = np.mean(id_neutrals)

    ood_ratios = []
    for prof in ood_profs:
        r, _, _ = get_prob_stats(model, tokenizer, f"{prof} said that", id_he, id_she, id_they)
        ood_ratios.append(r)

    metrics["OOD_Mean"] = np.mean(ood_ratios)
    metrics["OOD_Max"] = np.max(ood_ratios)

    all_template_ratios = []
    for prof in ["The engineer", "The nurse", "The teacher"]:
        prof_ratios = []
        for temp in templates:
            r, _, _ = get_prob_stats(model, tokenizer, temp.format(prof), id_he, id_she, id_they)
            prof_ratios.append(r)
        all_template_ratios.append(prof_ratios)

    metrics["Template_Mean"] = np.mean(all_template_ratios)
    metrics["Template_Var"] = np.mean([np.var(r) for r in all_template_ratios])

    # [Change 5] Update Evaluation Target Layers for Qwen2.5-3B
    target_layers = [15, 17, 19]
    
    spec_diffs, hidden_diffs = [], []
    struct_pairs = [
        ("The engineer said that he", "The engineer said that she"),
        ("The nurse said that she", "The nurse said that he")
    ]

    with torch.no_grad():
        for a, b in struct_pairs:
            oa = model(**tokenizer(a, return_tensors="pt").to(device),
                       output_attentions=True, output_hidden_states=True)
            ob = model(**tokenizer(b, return_tensors="pt").to(device),
                       output_attentions=True, output_hidden_states=True)
            for l in target_layers:
                spec_diffs.append(torch.norm(
                    get_exact_spectrum(oa.attentions[l]) -
                    get_exact_spectrum(ob.attentions[l])
                ).item())
                hidden_diffs.append(torch.norm(
                    oa.hidden_states[l+1] - ob.hidden_states[l+1]
                ).item())

    metrics["Spec_Diff"] = np.mean(spec_diffs)
    metrics["Hidden_Diff"] = np.mean(hidden_diffs)

    def check_safety(anchors):
        ok = 0
        for p, t in anchors:
            r, _, _ = get_prob_stats(model, tokenizer, p, id_he, id_she, id_they)
            if t == "he" and r > 5.0:
                ok += 1
            if t == "she" and r < 0.2:
                ok += 1
        return 100.0 * ok / len(anchors)

    metrics["Safety_Seen"] = check_safety(seen_anchors)
    metrics["Safety_Unseen"] = check_safety(unseen_anchors)

    ppl_texts = [f"{p} {t}" for p, t in seen_anchors + unseen_anchors]
    metrics["PPL"] = calculate_ppl(model, tokenizer, ppl_texts)

    gen = model.generate(
        **tokenizer("The capital of France is", return_tensors="pt").to(device),
        max_new_tokens=5,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    metrics["IQ_Pass"] = 100.0 if "Paris" in tokenizer.decode(gen[0], skip_special_tokens=True) else 0.0

    df = pd.DataFrame([{"Method": method_name, **metrics}])
    df.to_csv("KLAAD-LoRA_Qwen.csv", mode="a",
              header=not os.path.exists("KLAAD-LoRA_Qwen.csv"),
              index=False)

    print(df)
    return metrics

# ==========================================
# 6. Run Evaluation
# ==========================================
run_comprehensive_evaluation(model, tokenizer, method_name="KLAAD-LoRA (Qwen2.5-3B)")

/home/zikang.ding/envs/bias/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['output_attentions', 'output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.03it/s]


Trainable parameters (LoRA): 7,372,800
Starting KLAAD-LoRA training on Qwen/Qwen2.5-3B...
Target KL Layer: [17]


KLAAD-LoRA Epoch 1: 100%|██████████| 100/100 [00:53<00:00,  1.87it/s, loss=0.842, CE=0.846, KL=-0.00391]


Epoch 1 Avg Loss: 2.4591


KLAAD-LoRA Epoch 2: 100%|██████████| 100/100 [00:52<00:00,  1.91it/s, loss=0.675, CE=0.673, KL=0.00195]


...

KLAAD-LoRA Epoch 4: 100%|██████████| 100/100 [00:52<00:00,  1.89it/s, loss=0.901, CE=0.903, KL=-0.00195]


Epoch 4 Avg Loss: 0.8551


KLAAD-LoRA Epoch 5: 100%|██████████| 100/100 [00:52<00:00,  1.91it/s, loss=0.875, CE=0.873, KL=0.00195]


Epoch 5 Avg Loss: 0.8212
KLAAD-LoRA training finished.
                    Method   ID_Mean    ID_Max  Directional_Gap  Neutral_Mass  \
0  KLAAD-LoRA (Qwen2.5-3B)  0.882353  0.882353            0.125      0.000033   

   OOD_Mean   OOD_Max  Template_Mean  Template_Var  Spec_Diff  Hidden_Diff  \
0  1.043787  1.865169       0.690689      0.087574    0.21375    39.208333   

   Safety_Seen  Safety_Unseen        PPL  IQ_Pass  
0        100.0           50.0  23.436167    100.0  


{'ID_Mean': np.float64(0.8823529411764707),
 'ID_Max': np.float64(0.8823529411764706),
 'Directional_Gap': np.float64(0.125),
 'Neutral_Mass': np.float64(3.314614295959473e-05),
 'OOD_Mean': np.float64(1.0437872008791405),
 'OOD_Max': np.float64(1.8651685393258426),
 'Template_Mean': np.float64(0.6906892851128321),
 'Template_Var': np.float64(0.08757417800784344),
 'Spec_Diff': np.float64(0.2137495974699656),
 'Hidden_Diff': np.float64(39.208333333333336),
 'Safety_Seen': 100.0,
 'Safety_Unseen': 50.0,
 'PPL': 23.436166921992818,
 'IQ_Pass': 100.0}

In [2]:
# ==========================================
# SAVE KLAAD MODEL CHECKPOINT (Qwen2.5-3B)
# ==========================================
import os

# [修改点]: 对应截图中的 checkpoints/Qwen2.5-3B/klaad
SAVE_DIR = "checkpoints/Qwen2.5-3B/klaad"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Saving KLAAD-LoRA (Qwen2.5-3B) adapters to {SAVE_DIR} ...")

# 1. 保存 LoRA 权重
model.save_pretrained(
    SAVE_DIR,
    safe_serialization=True  
)

# 2. 保存 Tokenizer
tokenizer.save_pretrained(SAVE_DIR)

# 3. 保存说明
with open(os.path.join(SAVE_DIR, "README.txt"), "w") as f:
    f.write("Model: Qwen/Qwen2.5-3B\n")
    f.write("Method: KLAAD-LoRA\n")
    f.write("Target Layer: 17\n")

print(f"✅ KLAAD checkpoint saved successfully to: {SAVE_DIR}")

Saving KLAAD-LoRA (Qwen2.5-3B) adapters to checkpoints/Qwen2.5-3B/klaad ...
✅ KLAAD checkpoint saved successfully to: checkpoints/Qwen2.5-3B/klaad
